In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from os.path import abspath

# SparkSession
URL_SPARK = "spark://spark-master:7077"
warehouse_location = abspath('spark-warehouse')

spark = (
    SparkSession.builder
    .appName("read-stream-as-batch")
    .config("spark.executor.memory", "8g")
    .config("spark.executor.cores", "2")
    .config("spark.cores.max", "4")
    .config("spark.sql.warehouse.dir", warehouse_location)
    .config("spark.jars", "jars/hadoop-aws-3.3.2.jar,jars/aws-java-sdk-bundle-1.12.172.jar,jars/hadoop-common-3.3.2.jar,jars/spark-avro_2.12-3.3.2.jar,jars/commons-pool2-2.11.1.jar")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    .master(URL_SPARK)
    .getOrCreate()
)

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

/usr/local/lib/python3.9/dist-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


23/10/23 01:53:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/10/23 01:53:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp, from_unixtime

In [3]:
import os
os.environ["AWS_ACCESS_KEY_ID"] = "minio_access_key"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio_secret_key"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = f"http://minio:9000"

spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://minio:9000")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.access.key", "minio_access_key")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "minio_secret_key" )
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled","false")

In [4]:
path_to_stored_minio = "s3a://stream-interface-elab9070/data"

In [5]:
df_stream_to_batch = spark.read.parquet(path_to_stored_minio)

23/10/23 01:53:49 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [6]:
# display(df_stream_to_batch)

In [7]:
df_stream_to_batch.createTempView("myTable")

In [8]:
df_stream_to_batch.printSchema()

root
 |-- timestamp: string (nullable = true)
 |-- source: string (nullable = true)
 |-- interface_name: string (nullable = true)
 |-- packets_received: string (nullable = true)
 |-- packets_sent: string (nullable = true)
 |-- bytes_received: string (nullable = true)
 |-- bytes_sent: string (nullable = true)
 |-- multicast_packets_received: string (nullable = true)
 |-- multicast_packets_sent: string (nullable = true)
 |-- broadcast_packets_received: string (nullable = true)
 |-- broadcast_packets_sent: string (nullable = true)
 |-- output_drops: string (nullable = true)
 |-- output_queue_drops: string (nullable = true)
 |-- input_drops: string (nullable = true)
 |-- input_queue_drops: string (nullable = true)
 |-- input_errors: string (nullable = true)



In [14]:
df_stream_to_batch.show()

+--------------------+--------+--------------------+----------------+------------+--------------+-------------+--------------------------+----------------------+--------------------------+----------------------+------------+------------------+-----------+-----------------+------------+
|           timestamp|  source|      interface_name|packets_received|packets_sent|bytes_received|   bytes_sent|multicast_packets_received|multicast_packets_sent|broadcast_packets_received|broadcast_packets_sent|output_drops|output_queue_drops|input_drops|input_queue_drops|input_errors|
+--------------------+--------+--------------------+----------------+------------+--------------+-------------+--------------------------+----------------------+--------------------------+----------------------+------------+------------------+-----------+-----------------+------------+
|2023-10-05 20:10:...|elab9070|     Bundle-Ether101|               0|           0|             0|            0|                         0| 

In [9]:
# df_stream_to_batch.count()

In [10]:
sdf = spark.sql('SELECT MAX(cast(timestamp as timestamp)) FROM myTable')

In [11]:
sdf

max(CAST(timestamp AS TIMESTAMP))
2023-10-23 01:52:...


In [12]:
# sdf.show(2,truncate=False, vertical=True)

In [13]:
df_stream_to_batch.tail(5)

[Row(timestamp='2023-10-21 02:05:22.1', source='elab9070', interface_name='Bundle-Ether101', packets_received='0', packets_sent='0', bytes_received='0', bytes_sent='0', multicast_packets_received='0', multicast_packets_sent='0', broadcast_packets_received='0', broadcast_packets_sent='0', output_drops='0', output_queue_drops='0', input_drops='0', input_queue_drops='0', input_errors='0'),
 Row(timestamp='2023-10-11 09:58:01', source='elab9070', interface_name='Bundle-Ether101', packets_received='0', packets_sent='0', bytes_received='0', bytes_sent='0', multicast_packets_received='0', multicast_packets_sent='0', broadcast_packets_received='0', broadcast_packets_sent='0', output_drops='0', output_queue_drops='0', input_drops='0', input_queue_drops='0', input_errors='0'),
 Row(timestamp='2023-10-11 13:22:18', source='elab9070', interface_name='Bundle-Ether101', packets_received='0', packets_sent='0', bytes_received='0', bytes_sent='0', multicast_packets_received='0', multicast_packets_sent=